In [1]:
import numpy as np
import pandas as pd
import h5py
import keras
from keras.models import *
from keras.layers import *
from keras.optimizers import *
import matplotlib.pyplot as plt

import tensorflow as tf

from tensorflow.python.ops import nn

In [2]:
def loss_function_generator(y_pred, y_true):

    total_gen_loss = tf.reduce_mean(tf.abs(y_true - y_pred))
    
    return total_gen_loss

In [13]:
LR = 0.001
filters = [8, 16, 32, 32]
f1=4
f2=8
f3=16
ks=(5, 5)
channel_out = 3
chIn=3
input_size = (142,163,chIn)
inputs = Input(input_size)

initializer = tf.random_normal_initializer(0., 0.02)
kernel_initializer_N=initializer
activation_N='relu'

conv1 = keras.layers.Conv2D(filters = f1, kernel_size= ks, activation=activation_N,padding = 'same', kernel_initializer = kernel_initializer_N)(inputs)
pool1 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = keras.layers.Conv2D(filters = f2, kernel_size= ks, activation=activation_N,padding = 'same', kernel_initializer = kernel_initializer_N)(pool1)
pool2 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)
conv3 = keras.layers.Conv2D(filters = f3, kernel_size= ks, activation=activation_N,padding = 'same', kernel_initializer = kernel_initializer_N)(pool2)
pool3 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv3)
conv4 = keras.layers.Conv2D(filters = f3, kernel_size= ks, activation=activation_N,padding = 'same', kernel_initializer = kernel_initializer_N)(pool3)
pool4 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv4)
conv5 = keras.layers.Conv2D(filters = f3, kernel_size= ks, activation=activation_N,padding = 'same', kernel_initializer = kernel_initializer_N)(pool4)
pool5 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv5)
conv6 = keras.layers.Conv2D(filters = f2, kernel_size= ks, activation=activation_N,padding = 'same', kernel_initializer = kernel_initializer_N)(pool5)
pool6 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv6)
X = keras.layers.Flatten()(pool6)
X = keras.layers.Dense(512, activation='relu')(X)
X = keras.layers.Dense(1, activation='sigmoid')(X)
# Free up RAM in case the model definition cells were run multiple times
keras.backend.clear_session()

model = Model(inputs = inputs, outputs = X)

model.compile(optimizer = Adam(learning_rate = LR), loss = loss_function_generator, metrics = ['accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 142, 163, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 142, 163, 4)       304       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 71, 81, 4)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 71, 81, 8)         808       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 35, 40, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 35, 40, 16)        3216      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 20, 16)        0     